In [1]:
import os
import pymysql
from tqdm import tqdm
from dotenv import load_dotenv

pymysql.install_as_MySQLdb()

## Connect DB

In [2]:
load_dotenv()
ENDPOINT = os.environ.get("dev_db_endpoint")
USER = os.environ.get("dev_db_user")
PASSWORD = os.environ.get("dev_db_password")

In [3]:
conn = pymysql.connect(
    host=ENDPOINT,
    user=USER,
    password=PASSWORD,
    database="aresa",
    port=3306
)

In [4]:
cur = conn.cursor()

In [5]:
from sqlalchemy import create_engine
import sqlalchemy

In [11]:
engine = create_engine(f"mysql+mysqldb://{USER}:{PASSWORD}@{ENDPOINT}:3306/aresa")
# http://i5on9i.blogspot.com/2020/05/no-module-named-mysqldb.html

## Insert Data

In [12]:
import pandas as pd

In [24]:
collect_df = pd.read_csv("config/한국은행_수집_리스트.csv")
df_all = pd.DataFrame()

for row in tqdm(collect_df.itertuples()):
    data_nm = row.data_nm
    code = row.code.lower()
    cycle = row.cycle.lower()
    search_start = row.search_start
    search_end = row.search_end

    df = pd.read_csv(f"data/한국은행/{data_nm}_{code}_{cycle}_{search_start}_{search_end}.csv")
    df["cycle"] = cycle

    df_all = pd.concat([df_all, df])

    # df.to_sql(f'bok_{code}_{cycle}', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

22it [00:01, 14.71it/s]


In [25]:
df_all.columns = [column.lower() for column in df_all.columns]

In [26]:
df_all.info()
df_all.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 664630 entries, 0 to 10385
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   stat_code   664630 non-null  object 
 1   stat_name   664630 non-null  object 
 2   item_code1  664630 non-null  object 
 3   item_name1  664630 non-null  object 
 4   item_code2  18330 non-null   object 
 5   item_name2  18330 non-null   object 
 6   item_code3  3456 non-null    float64
 7   item_name3  3456 non-null    object 
 8   item_code4  0 non-null       float64
 9   item_name4  0 non-null       float64
 10  unit_name   661174 non-null  object 
 11  time        664630 non-null  object 
 12  data_value  664519 non-null  float64
 13  cycle       664630 non-null  object 
dtypes: float64(4), object(10)
memory usage: 76.1+ MB


,stat_code,stat_name,item_code1,item_name1,item_code2,item_name2,item_code3,item_name3,item_code4,item_name4,unit_name,time,data_value,cycle
0,802Y001,1.5.1.1. 주식시장(일),1000,KOSPI지수,NaN,NaN,NaN,NaN,NaN,NaN,1980.01.04=100,19950103,1013.57,d
1,802Y001,1.5.1.1. 주식시장(일),1000,KOSPI지수,NaN,NaN,NaN,NaN,NaN,NaN,1980.01.04=100,19950104,997.01,d
2,802Y001,1.5.1.1. 주식시장(일),1000,KOSPI지수,NaN,NaN,NaN,NaN,NaN,NaN,1980.01.04=100,19950105,976.06,d
3,802Y001,1.5.1.1. 주식시장(일),1000,KOSPI지수,NaN,NaN,NaN,NaN,NaN,NaN,1980.01.04=100,19950106,975.32,d
4,802Y001,1.5.1.1. 주식시장(일),1000,KOSPI지수,NaN,NaN,NaN,NaN,NaN,NaN,1980.01.04=100,19950107,988.80,d


In [27]:
df_all.to_sql('bok_all_rawdata', engine, schema="aresa", index=False, if_exists='replace', chunksize=100000)

664630